# Crawling Coursera - Deeplearning specialization

## 1.Connect to Database

In [2]:
import mysql
import mysql.connector
# run this every time
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="pdne",
)
mycursor = mydb.cursor()
# mycursor.execute("CREATE DATABASE Coursera")


In [3]:
mycursor.execute("USE Coursera")
mycursor.execute('DROP TABLE Syllabus')
mycursor.execute('DROP TABLE Deeplearning')

In [4]:
query = '''
CREATE TABLE Deeplearning 
(id INT AUTO_INCREMENT PRIMARY KEY, 
name VARCHAR(511),
course_no INT,
stars FLOAT,
ratings INT,
enrolled INT,
week_count INT)
'''
mycursor.execute(query)

In [5]:
query = '''
CREATE TABLE Syllabus 
(id INT AUTO_INCREMENT PRIMARY KEY, 
course_id INT,
course_name VARCHAR(255),
week_no INT,
week_name VARCHAR(255),
duration FLOAT,
FOREIGN KEY (course_id) REFERENCES Deeplearning(id))
'''
mycursor.execute(query)

In [6]:
mycursor.execute("SHOW TABLES")
for table in mycursor:
    print(table)

('Deeplearning',)
('Syllabus',)


## 2.Crawl Coursera

In [7]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver import Firefox
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pyperclip

path_to_geckodriver = '/Users/ngn/geckodriver'
service = Service(path_to_geckodriver)
driver = Firefox(service=service)

# navigate to temp mail webpage
def add_course(course_no,url):

    # add course
    driver.get(url)
    stars = driver.find_element_by_xpath('/html/body/div[2]/div/div/main/div/div[1]/div[1]/div[1]/div/div/div[1]/div[3]/div/div/div[1]/ul/li/a/div/span')
    ratings = driver.find_element_by_xpath('/html/body/div[2]/div/div/main/div/div[1]/div[1]/div[1]/div/div/div[1]/div[3]/div/div/div[1]/ul/li/a/div/div[2]/span/span')
    course_name = driver.find_element_by_xpath('/html/body/div[2]/div/div/main/div/div[1]/div[1]/div[1]/div/div/div[1]/div[2]/h1')
    week_count = len(driver.find_elements_by_css_selector("div._jyhj5r.SyllabusWeek"))
    enrolled = driver.find_element_by_xpath('/html/body/div[2]/div/div/main/div/div[1]/div[1]/div[1]/div/div/div[1]/div[5]/div/div[2]/div/div/span/strong/span')

    stars = float(stars.text[0:3])
    ratings = int(ratings.text.split(' ')[0].replace(',','_'))
    course_name = course_name.text
    enrolled = int(enrolled.text.replace(',','_'))
    
    sql = "INSERT INTO Deeplearning (name, course_no, stars, ratings, enrolled, week_count) VALUES (%s, %s, %s,%s, %s, %s)"
    val = (course_name, course_no, stars, ratings, enrolled, week_count)
    mycursor.execute(sql, val)
    mydb.commit()

    # add syllabus
    weeks = driver.find_elements_by_css_selector("div._jyhj5r.SyllabusWeek")
    for i in range(len(weeks)):
        css_selector_name = f'div.SyllabusWeek:nth-child({i+1}) > div:nth-child(3) > div:nth-child(1) > h3:nth-child(2)'
        css_selector_week = f'div.SyllabusWeek:nth-child({i+1}) > div:nth-child(1) > div:nth-child(1) > span:nth-child(2)'
        css_selector_time = f'div.SyllabusWeek:nth-child({i+1}) > div:nth-child(3) > div:nth-child(1) > div:nth-child(5) > div:nth-child(1) > div:nth-child(2) > span:nth-child(1) > span:nth-child(1) > span:nth-child(1)'
        name = driver.find_elements_by_css_selector(css_selector_name)[0].text
        week_no = int(driver.find_elements_by_css_selector(css_selector_week)[0].text)
        duration = driver.find_elements_by_css_selector(css_selector_time)[0].text
        duration = int(duration.split(' ')[1])
        
        sql = "INSERT INTO Syllabus (course_id,course_name,week_name, week_no, duration) VALUES (%s, %s,%s, %s, %s)"
        val = (course_no,course_name,name, week_no, duration)   # warning : forien key is not course_id.but here is ok!
        mycursor.execute(sql, val)
        mydb.commit()   

In [8]:
courses_url = [
    'https://www.coursera.org/learn/neural-networks-deep-learning?specialization=deep-learning',
    'https://www.coursera.org/learn/deep-neural-network?specialization=deep-learning',
    'https://www.coursera.org/learn/machine-learning-projects?specialization=deep-learning',
    'https://www.coursera.org/learn/convolutional-neural-networks?specialization=deep-learning',
    'https://www.coursera.org/learn/nlp-sequence-models?specialization=deep-learning'
]

for i in range(len(courses_url)):
    add_course(i+1,courses_url[i])

driver.quit()

/var/folders/bf/9616ns414yg2qn2p8czl2sr00000gn/T/ipykernel_36817/1429135153.py:21: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  stars = driver.find_element_by_xpath('/html/body/div[2]/div/div/main/div/div[1]/div[1]/div[1]/div/div/div[1]/div[3]/div/div/div[1]/ul/li/a/div/span')
/var/folders/bf/9616ns414yg2qn2p8czl2sr00000gn/T/ipykernel_36817/1429135153.py:22: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  ratings = driver.find_element_by_xpath('/html/body/div[2]/div/div/main/div/div[1]/div[1]/div[1]/div/div/div[1]/div[3]/div/div/div[1]/ul/li/a/div/div[2]/span/span')
/var/folders/bf/9616ns414yg2qn2p8czl2sr00000gn/T/ipykernel_36817/1429135153.py:23: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  course_name = driver.find_element_by_xpath('/html/body/div[2]/div/div/main/div/div[1]/div[1]/div[1]/div/div/div[1]/div[2]/h1')
/var/folders/

## Deeplearning Spetialization Courses Details

In [9]:
import pandas as pd
query = 'SELECT * FROM Deeplearning'
df = pd.read_sql(query, mydb)
df

,id,name,course_no,stars,ratings,enrolled,week_count
0,1,Neural Networks and Deep Learning,1,4.9,117889,1169767,4
1,2,Improving Deep Neural Networks: Hyperparameter...,2,4.9,61910,487064,3
2,3,Structuring Machine Learning Projects,3,4.8,48981,407755,2
3,4,Convolutional Neural Networks,4,4.9,41317,446422,4
4,5,Sequence Models,5,4.8,29178,360416,4


### Syllabus of courses above

In [10]:
import pandas as pd
query = 'SELECT * FROM Syllabus'
df = pd.read_sql(query, mydb)
df

,id,course_id,course_name,week_no,week_name,duration
0,1,1,Neural Networks and Deep Learning,1,Introduction to Deep Learning,74.0
1,2,1,Neural Networks and Deep Learning,2,Neural Networks Basics,161.0
2,3,1,Neural Networks and Deep Learning,3,Shallow Neural Networks,109.0
3,4,1,Neural Networks and Deep Learning,4,Deep Neural Networks,64.0
4,5,2,Improving Deep Neural Networks: Hyperparameter...,1,Practical Aspects of Deep Learning,131.0
5,6,2,Improving Deep Neural Networks: Hyperparameter...,2,Optimization Algorithms,92.0
6,7,2,Improving Deep Neural Networks: Hyperparameter...,3,"Hyperparameter Tuning, Batch Normalization and...",103.0
7,8,3,Structuring Machine Learning Projects,1,ML Strategy,100.0
8,9,3,Structuring Machine Learning Projects,2,ML Strategy,132.0
9,10,4,Convolutional Neural Networks,1,Foundations of Convolutional Neural Networks,140.0
